In [1]:
import numpy as np

In [28]:
def smith_waterman(seq1, seq2, match_award=1, gap_penalty=-1, mismatch_penalty=-1):

    def match_score(a, b):
        if a == b:
            return match_award
        elif a == '-' or b == '-':
            return gap_penalty
        else:
            return mismatch_penalty

    n = len(seq1)
    m = len(seq2)  
    
    score = np.zeros((m+1, n+1)).astype(int)
   
    for i in range(0, m + 1):
        score[i,0] = 0
    
    for j in range(0, n + 1):
        score[0,j] = 0
    
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            diag = score[i - 1,j - 1] + match_score(seq1[j-1], seq2[i-1])
            up = score[i - 1,j] + gap_penalty
            left = score[i,j - 1] + gap_penalty
            score[i,j] = max(diag, left, up, 0)
            
    i, j = np.unravel_index(np.argmax(score), score.shape)
    trace1 = []
    trace2 = []
    m = 1
    while i > 0 or j > 0:
        diag = score[i - 1,j - 1] + match_score(seq1[j-1], seq2[i-1])
        up = score[i - 1,j] + gap_penalty
        left = score[i,j - 1] + gap_penalty
#         print((diag, up, left))

        if not (i > 0 and j > 0):
            diag = - np.inf
        if not (j > 0):
            left = - np.inf
        if not (i > 0):
            up = - np.inf
        
        m = max(diag, left, up, 0)
        
        if m == 0: break
        
        if diag == m:
            trace1.append(seq1[j-1])
            trace2.append(seq2[i-1])
            i -= 1
            j -= 1
        elif left == m:
            trace1.append(seq1[j-1])
            trace2.append("-")
            j -= 1
        elif up == m:
            trace1.append("-")
            trace2.append(seq2[i-1])
            i -= 1

    return score, (trace1[::-1], trace2[::-1])

In [46]:
match_award=2
gap_penalty=-2
mismatch_penalty=-1

seq1 = "GAATTCAGTTA"
seq2 = "GGATCGA"

matrix, traces = smith_waterman(seq1, seq2, match_award, gap_penalty, mismatch_penalty)

print(matrix)
print(traces[0])
print(traces[1])

[[0 0 0 0 0 0 0 0 0 0 0 0]
 [0 2 0 0 0 0 0 0 2 0 0 0]
 [0 2 1 0 0 0 0 0 2 1 0 0]
 [0 0 4 3 1 0 0 2 0 1 0 2]
 [0 0 2 3 5 3 1 0 1 2 3 1]
 [0 0 0 1 3 4 5 3 1 0 1 2]
 [0 2 0 0 1 2 3 4 5 3 1 0]
 [0 0 4 2 0 0 1 5 3 4 2 3]]
['G', 'A', 'A', 'T']
['G', 'G', 'A', 'T']


In [47]:
np.where(matrix == np.max(matrix))

(array([4, 5, 6, 7], dtype=int64), array([4, 6, 8, 7], dtype=int64))

In [31]:
import swalign

In [48]:
# choose your own values here… 2 and -1 are common.
match = 2
mismatch = -1
scoring = swalign.NucleotideScoringMatrix(match, mismatch)

sw = swalign.LocalAlignment(scoring, gap_penalty=-2)  # you can also choose gap penalties, etc...
alignment = sw.align(seq1,seq2)
alignment.dump()

Query: 1 GGATCGA 7
         |.||..|
Ref  : 1 GAATTCA 7

Score: 5
Matches: 4 (57.1%)
Mismatches: 3
CIGAR: 7M


In [35]:
?swalign.NucleotideScoringMatrix